In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,71,2024-03-24,1,115.757143,97.325714,0.560186,11.450000,25.188571,0.204786,118.820000,...,13.162857,24.642857,0.196286,111.524286,0,3,6,2,19,2024
1,71,2024-03-24,-2,114.357143,97.000000,0.528357,10.010000,24.554286,0.227443,117.482857,...,11.334783,24.137681,0.216449,120.617391,0,3,6,2,23,2024
2,71,2024-03-24,-2,113.057143,96.447143,0.556057,11.562857,23.478571,0.184500,116.418571,...,10.964286,21.738571,0.213314,114.292857,0,3,6,2,6,2024
3,70,2024-03-24,4,120.739130,99.091304,0.576565,10.469565,20.889855,0.202406,121.134783,...,10.878571,21.555714,0.219400,120.388571,0,3,6,2,21,2024
4,70,2024-03-24,-1,118.463768,99.026087,0.554319,11.839130,27.502899,0.178435,118.624638,...,12.288571,23.280000,0.212729,116.724286,0,3,6,2,10,2024
5,72,2024-03-24,2,122.816901,101.276056,0.578310,10.981690,23.271831,0.177648,121.078873,...,11.828571,19.535714,0.212057,116.435714,0,3,6,2,12,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1
1,2024-03-24,PHI,LAC,4.500000,1.219780,0
2,2024-03-24,CLE,MIA,2.450000,1.571429,0
3,2024-03-24,OKC,MIL,2.240000,1.675676,0
4,2024-03-24,GSW,MIN,2.200000,1.704225,0
5,2024-03-24,IND,LAL,2.300000,1.645161,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1
1,2024-03-24,PHI,LAC,4.500000,1.219780,0
2,2024-03-24,CLE,MIA,2.450000,1.571429,1
3,2024-03-24,OKC,MIL,2.240000,1.675676,0
4,2024-03-24,GSW,MIN,2.200000,1.704225,0
5,2024-03-24,IND,LAL,2.300000,1.645161,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1
1,2024-03-24,PHI,LAC,4.500000,1.219780,0
2,2024-03-24,CLE,MIA,2.450000,1.571429,0
3,2024-03-24,OKC,MIL,2.240000,1.675676,0
4,2024-03-24,GSW,MIN,2.200000,1.704225,0
5,2024-03-24,IND,LAL,2.300000,1.645161,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1
1,2024-03-24,PHI,LAC,4.500000,1.219780,0
2,2024-03-24,CLE,MIA,2.450000,1.571429,0
3,2024-03-24,OKC,MIL,2.240000,1.675676,0
4,2024-03-24,GSW,MIN,2.200000,1.704225,0
5,2024-03-24,IND,LAL,2.300000,1.645161,1


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1.0
1,2024-03-24,PHI,LAC,4.500000,1.219780,0.0
2,2024-03-24,CLE,MIA,2.450000,1.571429,1.0
3,2024-03-24,OKC,MIL,2.240000,1.675676,1.0
4,2024-03-24,GSW,MIN,2.200000,1.704225,1.0
5,2024-03-24,IND,LAL,2.300000,1.645161,1.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-24,NOP,DET,1.142857,6.000000,1.0
1,2024-03-24,PHI,LAC,4.500000,1.219780,0.0
2,2024-03-24,CLE,MIA,2.450000,1.571429,1.0
3,2024-03-24,OKC,MIL,2.240000,1.675676,1.0
4,2024-03-24,GSW,MIN,2.200000,1.704225,1.0
5,2024-03-24,IND,LAL,2.300000,1.645161,1.0
